# Практическое задание к уроку 5. Part-of-Speech разметка, NER, извлечение отношений


Задание 1. Написать теггер на данных с русским языком

1.	проверить UnigramTagger, BigramTagger, TrigramTagger и их комбмнации

2.	написать свой теггер как на занятии, попробовать разные векторайзеры, добавить знание не только букв но и слов

3.	сравнить все реализованные методы сделать выводы


Задание 2. Проверить насколько хорошо работает NER

данные брать из http://www.labinform.ru/pub/named_entities/

1.	проверить NER из nltk/spacy/deeppavlov

2.	написать свой нер попробовать разные подходы

    a.	передаём в сетку токен и его соседей
    
    b.	передаём в сетку только токен
    
    c.	свой вариант


3.	сравнить ваши реализованные подходы на качество (вывести precision/recall/f1_score)

## Задание 1. Написать теггер на данных с русским языком

### 1.1. Проверить UnigramTagger, BigramTagger, TrigramTagger и их комбмнации

In [1]:
!pip install pyconll

In [2]:
!pip install sklearn_crfsuite

In [3]:
import pandas as pd
import numpy as np
import re
import string

from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.feature_extraction.text import CountVectorizer, HashingVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

import pyconll
import nltk
from nltk.tag import DefaultTagger, UnigramTagger, BigramTagger, TrigramTagger

import sklearn
import sklearn_crfsuite
from sklearn_crfsuite import scorers
from sklearn_crfsuite import metrics
from sklearn.metrics import f1_score

In [4]:
# Скачивание данных

# !wget -O ru_syntagrus-ud-train.conllu https://raw.githubusercontent.com/UniversalDependencies/UD_Russian-SynTagRus/master/ru_syntagrus-ud-train.conllu
!wget -O ru_syntagrus-ud-train.conllu https://raw.githubusercontent.com/UniversalDependencies/UD_Russian-SynTagRus/master/ru_syntagrus-ud-train-c.conllu
!wget -O ru_syntagrus-ud-dev.conllu https://raw.githubusercontent.com/UniversalDependencies/UD_Russian-SynTagRus/master/ru_syntagrus-ud-dev.conllu

--2022-06-16 20:02:30--  https://raw.githubusercontent.com/UniversalDependencies/UD_Russian-SynTagRus/master/ru_syntagrus-ud-train-c.conllu
Распознаётся raw.githubusercontent.com (raw.githubusercontent.com)… 185.199.109.133, 185.199.110.133, 185.199.111.133, ...
Подключение к raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... соединение установлено.
HTTP-запрос отправлен. Ожидание ответа… 200 OK
Длина: 32367464 (31M) [text/plain]
Сохранение в: «ru_syntagrus-ud-train.conllu»

ru_syntagrus-ud-tra 100%[===================>]  30.87M  22.2MB/s    за 1.4s    

2022-06-16 20:02:33 (22.2 MB/s) - «ru_syntagrus-ud-train.conllu» сохранён [32367464/32367464]

--2022-06-16 20:02:33--  https://raw.githubusercontent.com/UniversalDependencies/UD_Russian-SynTagRus/master/ru_syntagrus-ud-dev.conllu
Распознаётся raw.githubusercontent.com (raw.githubusercontent.com)… 185.199.110.133, 185.199.111.133, 185.199.108.133, ...
Подключение к raw.githubusercontent.com (raw.githubusercon

In [5]:
# Загрузка данных

full_train = pyconll.load_from_file('ru_syntagrus-ud-train.conllu')
full_test = pyconll.load_from_file('ru_syntagrus-ud-dev.conllu')

In [6]:
for sent in full_train[:2]:
    for token in sent:
        print(token.form, token.upos)
    print()

Моя DET
мать NOUN
, PUNCT
Анна PROPN
Всеволодовна PROPN
Мохова PROPN
( PUNCT
Дмитриева PROPN
) PUNCT
, PUNCT
родилась VERB
27 ADJ
марта NOUN
1913 ADJ
года NOUN
. PUNCT

Отец NOUN
ее DET
, PUNCT
Всеволод PROPN
Нестерович PROPN
Дмитриев PROPN
, PUNCT
служил VERB
тогда ADV
егерем NOUN
при ADP
императорской ADJ
охотничьей ADJ
усадьбе NOUN
в ADP
Карелии PROPN
. PUNCT



In [7]:
MAX_SENT_LEN = max(len(sent) for sent in full_train)
# MAX_ORIG_TOKEN_LEN = max(len(token.form) for sent in full_train for token in sent)
MAX_ORIG_TOKEN_LEN = max(len(token.form) for sent in full_train)
print('Наибольшая длина предложения', MAX_SENT_LEN)
print('Наибольшая длина токена', MAX_ORIG_TOKEN_LEN)

Наибольшая длина предложения 98
Наибольшая длина токена 1


In [8]:
# Заполним данными
fdata_train = []
for sent in full_train[:]:
    fdata_train.append([(token.form, token.upos) for token in sent])
    
fdata_test = []
for sent in full_test[:]:
    fdata_test.append([(token.form, token.upos) for token in sent])
    
fdata_sent_test = []
for sent in full_test[:]:
    fdata_sent_test.append([token.form for token in sent])

In [9]:
# Проверим UnigramTagger, BigramTagger, TrigramTagger и их комбинации

unigram_tagger = UnigramTagger(fdata_train)
unigram_acc = unigram_tagger.evaluate(fdata_test)

bigram_tagger = BigramTagger(fdata_train)
bigram_acc = bigram_tagger.evaluate(fdata_test)

trigram_tagger = TrigramTagger(fdata_train)
trigram_acc = trigram_tagger.evaluate(fdata_test)

bigram_tagger = BigramTagger(fdata_train, backoff=unigram_tagger)
bigram_unigram_acc = bigram_tagger.evaluate(fdata_test)

trigram_tagger = TrigramTagger(fdata_train, backoff=bigram_tagger)
trigram_bigram_unigram_acc = trigram_tagger.evaluate(fdata_test)

print(f'Accuracy:\nUnigram Tagger: {round(unigram_acc, 3)},\nBigram Tagger: {round(bigram_acc, 3)},\n'
      f'Trigram Tagger: {round(trigram_acc, 3)},\nBigram and Unigram Tagger: {round(bigram_unigram_acc, 3)},\n'
      f'Trigram, Bigram and Unigram Tagger: {round(trigram_bigram_unigram_acc, 3)},\n')

Accuracy:
Unigram Tagger: 0.801,
Bigram Tagger: 0.581,
Trigram Tagger: 0.333,
Bigram and Unigram Tagger: 0.806,
Trigram, Bigram and Unigram Tagger: 0.804,



### 1.2. Написать свой теггер как на занятии, попробовать разные векторайзеры, добавить знание не только букв но и слов

In [10]:
from sklearn.feature_extraction.text import TfidfVectorizer, HashingVectorizer, CountVectorizer
# from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder

In [11]:
# Переведём тренировочный датасет в списки слов и списки POS-разметки
train_tok = []
train_label = []
for sent in fdata_train[:]:
    for tok in sent:
        train_tok.append(tok[0])
        train_label.append('NO_TAG' if tok[1] is None else tok[1])
        
test_tok = []
test_label = []
for sent in fdata_test[:]:
    for tok in sent:
        test_tok.append(tok[0])
        test_label.append('NO_TAG' if tok[1] is None else tok[1])

In [12]:
train_tok[:10], train_label[:10]

(['Моя',
  'мать',
  ',',
  'Анна',
  'Всеволодовна',
  'Мохова',
  '(',
  'Дмитриева',
  ')',
  ','],
 ['DET',
  'NOUN',
  'PUNCT',
  'PROPN',
  'PROPN',
  'PROPN',
  'PUNCT',
  'PROPN',
  'PUNCT',
  'PUNCT'])

In [13]:
le = LabelEncoder()
train_enc_labels = le.fit_transform(train_label)
train_enc_labels

array([ 5,  7, 13, ...,  0,  7, 13])

In [14]:
test_enc_labels = le.transform(test_label)
test_enc_labels

array([ 7, 13,  1, ...,  0,  7, 13])

In [15]:
le.classes_

array(['ADJ', 'ADP', 'ADV', 'AUX', 'CCONJ', 'DET', 'INTJ', 'NOUN',
       'NO_TAG', 'NUM', 'PART', 'PRON', 'PROPN', 'PUNCT', 'SCONJ', 'SYM',
       'VERB', 'X'], dtype='<U6')

### 1.3. Сравнить все реализованные методы сделать выводы

In [16]:
for vectorizer in [CountVectorizer, HashingVectorizer, TfidfVectorizer]:

    scaler = StandardScaler(with_mean=False)
    coder = vectorizer(ngram_range=(1, 5), analyzer='char')
    

    X_train = coder.fit_transform(train_tok)
    X_test = coder.transform(test_tok)
    
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.fit_transform(X_test)    
    
    lr = LogisticRegression(random_state=0, max_iter = 100, n_jobs=7)
    lr.fit(X_train, train_enc_labels)

    pred = lr.predict(X_test)

    print(vectorizer, accuracy_score(test_enc_labels, pred))

AttributeError: 'NoneType' object has no attribute 'lower'

## Задание 2. Проверить насколько хорошо работает NER

данные брать из http://www.labinform.ru/pub/named_entities/

### 2.1. Проверить NER из nltk/spacy/deeppavlov

### 2.2. Написать свой нер попробовать разные подходы

a. передаём в сетку токен и его соседей

b. передаём в сетку только токен

c. свой вариант


### 2.3. Сравнить ваши реализованные подходы на качество (вывести precision/recall/f1_score)

In [17]:
!pip install corus

In [18]:
import corus
from corus import load_ne5
import nltk

In [19]:
nltk.download('maxent_ne_chunker')
nltk.download('words')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     /Users/alenakukhta/nltk_data...
[nltk_data]   Package maxent_ne_chunker is already up-to-date!
[nltk_data] Downloading package words to
[nltk_data]     /Users/alenakukhta/nltk_data...
[nltk_data]   Package words is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/alenakukhta/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/alenakukhta/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [20]:
!wget http://www.labinform.ru/pub/named_entities/collection5.zip

--2022-06-16 20:03:22--  http://www.labinform.ru/pub/named_entities/collection5.zip
Распознаётся www.labinform.ru (www.labinform.ru)… 95.181.230.181
Подключение к www.labinform.ru (www.labinform.ru)|95.181.230.181|:80... соединение установлено.
HTTP-запрос отправлен. Ожидание ответа… 200 OK
Длина: 1899530 (1.8M) [application/zip]
Сохранение в: «collection5.zip.3»

collection5.zip.3   100%[===================>]   1.81M  --.-KB/s    за 0.1s    

2022-06-16 20:03:22 (13.3 MB/s) - «collection5.zip.3» сохранён [1899530/1899530]



In [21]:
# !unzip collection5.zip

records = load_ne5('Collection5/')
records

<generator object load_ne5 at 0x7f95871ffcf0>

In [22]:
document = next(records).text
document

'Жириновский предлагает обменять с США Сноудена на Бута\r\n\r\nЛидер ЛДПР Владимир Жириновский предложил обменять бывшего сотрудника ЦРУ США Эдварда Сноудена, который прибыл в Москву, на осужденного в Америке бизнесмена Виктора Бута.\r\n\r\n"Сноудена ни в коем случае не высылать в США, а обменять на Виктора Бута и Константина Ярошенко. В идеале — добавить генерала Олега Калугина", — написал он в своем микроблоге в Twitter.\r\n\r\nСноуден, работавший на компанию Booz Allen Hamilton — подрядчика Центрального разведывательного управления США, в начале июня распространил секретный ордер суда, по которому спецслужбы получили доступ ко всем звонкам крупнейшего сотового оператора Verizon, а также данные о сверхсекретной программе агентства национальной безопасности PRISM, позволяющей отслеживать электронные коммуникации на крупнейших сайтах. В воскресенье стало известно, что Сноуден прибыл из Гонконга в Москву и запросил убежища в Эквадоре.\r\n\r\nЧто ждет Эдварда Сноудена\r\n\r\nЭдвард Сноуд

In [23]:
{(' '.join(c[0] for c in chunk), chunk.label() ) for chunk in nltk.ne_chunk(nltk.pos_tag(nltk.word_tokenize(document))) if hasattr(chunk, 'label') }

{('Hamilton', 'PERSON'),
 ('Америке', 'PERSON'),
 ('Виктора Бута', 'PERSON'),
 ('Москву', 'PERSON'),
 ('Сноуден', 'PERSON'),
 ('Эдварда Сноудена', 'PERSON'),
 ('Эдварда Сноудена Эдвард Сноуден', 'PERSON')}

**Вывод**: Качество плохое.

## 2.1. Проверить NER из nltk/spacy/deeppavlov

In [24]:
!pip uninstall -y tensorflow tensorflow-gpu
!pip install numpy scipy librosa unidecode inflect librosa transformers

Found existing installation: tensorflow 2.9.1
Uninstalling tensorflow-2.9.1:
  Successfully uninstalled tensorflow-2.9.1


In [25]:
import numpy

In [26]:
!pip install virtualenv

In [27]:
!python -m venv env 
#.\env\Scripts\activate.bat

In [28]:
!python -m venv env.\env\Scripts\activate.bat

In [29]:
import sys

In [30]:
!{sys.executable} -m pip install --user deeppavlov

  Using cached deeppavlov-0.17.4-py3-none-any.whl (878 kB)
  Using cached scipy-1.4.1.tar.gz (24.6 MB)
  Installing build dependencies ... error
  error: subprocess-exited-with-error
  
  × pip subprocess to install build dependencies did not run successfully.
  │ exit code: 1
  ╰─> [2480 lines of output]
      Ignoring numpy: markers 'python_version == "3.5" and platform_system != "AIX"' don't match your environment
      Ignoring numpy: markers 'python_version == "3.6" and platform_system != "AIX"' don't match your environment
      Ignoring numpy: markers 'python_version == "3.7" and platform_system != "AIX"' don't match your environment
      Ignoring numpy: markers 'python_version == "3.5" and platform_system == "AIX"' don't match your environment
      Ignoring numpy: markers 'python_version == "3.6" and platform_system == "AIX"' don't match your environment
      Ignoring numpy: markers 'python_version == "3.7" and platform_system == "AIX"' don't match your environment
      Ign

In [31]:
!pip install deeppavlov

  Using cached deeppavlov-0.17.4-py3-none-any.whl (878 kB)
  Using cached Cython-0.29.14.tar.gz (2.1 MB)
  Preparing metadata (setup.py) ... done
  Using cached filelock-3.0.12-py3-none-any.whl (7.6 kB)
  Using cached scikit-learn-0.21.2.tar.gz (12.2 MB)
  Preparing metadata (setup.py) ... done
  Using cached numpy-1.18.0.zip (5.4 MB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached pydantic-1.3-py36.py37.py38-none-any.whl (85 kB)
  Using cached overrides-2.7.0.tar.gz (4.5 kB)
  Preparing metadata (setup.py) ... done
  Using cached pymorphy2-0.8-py2.py3-none-any.whl (46 kB)
  Using cached fastapi-0.47.1-py3-none-any.whl (43 kB)
  Using cached pyTelegramBotAPI-3.6.7.tar.gz (65 kB)
  Preparing metadata (setup.py) ... done
  Using cached pandas-0.25.3.tar.gz (12.6 MB)
  Preparing metadata (setup.py) ... done
  Using cached h5py-2.10.0.tar.gz (301 kB)
  Preparing metadata (setup.py) ... don

            removing: _configtest.c _configtest.o
            C compiler: clang -Wno-unused-result -Wsign-compare -Wunreachable-code -DNDEBUG -fwrapv -O2 -Wall -fPIC -O2 -isystem /Users/alenakukhta/opt/anaconda3/include -arch x86_64 -I/Users/alenakukhta/opt/anaconda3/include -fPIC -O2 -isystem /Users/alenakukhta/opt/anaconda3/include -arch x86_64
      
            compile options: '-Inumpy/core/src/common -Inumpy/core/src -Inumpy/core -Inumpy/core/src/npymath -Inumpy/core/src/multiarray -Inumpy/core/src/umath -Inumpy/core/src/npysort -I/Users/alenakukhta/opt/anaconda3/include/python3.9 -c'
            clang: _configtest.c
            _configtest.c:5:20: error: unknown attribute 'optimize' ignored [-Werror,-Wunknown-attributes]
            int __attribute__((optimize("O3"))) attribute_optimize_opt_3(void*);
                               ^
            1 error generated.
            failure.
            removing: _configtest.c _configtest.o
            C compiler: clang -Wno-unused-resu

In [32]:
!python -m deeppavlov install squad_bert
!python -m deeppavlov install ner_ontonotes

/Users/alenakukhta/opt/anaconda3/bin/python: No module named deeppavlov
/Users/alenakukhta/opt/anaconda3/bin/python: No module named deeppavlov


In [33]:
import deeppavlov
from deeppavlov import configs, build_model

ModuleNotFoundError: No module named 'deeppavlov'

In [34]:
document

'Жириновский предлагает обменять с США Сноудена на Бута\r\n\r\nЛидер ЛДПР Владимир Жириновский предложил обменять бывшего сотрудника ЦРУ США Эдварда Сноудена, который прибыл в Москву, на осужденного в Америке бизнесмена Виктора Бута.\r\n\r\n"Сноудена ни в коем случае не высылать в США, а обменять на Виктора Бута и Константина Ярошенко. В идеале — добавить генерала Олега Калугина", — написал он в своем микроблоге в Twitter.\r\n\r\nСноуден, работавший на компанию Booz Allen Hamilton — подрядчика Центрального разведывательного управления США, в начале июня распространил секретный ордер суда, по которому спецслужбы получили доступ ко всем звонкам крупнейшего сотового оператора Verizon, а также данные о сверхсекретной программе агентства национальной безопасности PRISM, позволяющей отслеживать электронные коммуникации на крупнейших сайтах. В воскресенье стало известно, что Сноуден прибыл из Гонконга в Москву и запросил убежища в Эквадоре.\r\n\r\nЧто ждет Эдварда Сноудена\r\n\r\nЭдвард Сноуд

In [35]:
deeppavlov_ner = build_model(configs.ner.ner_bert_ent_and_type_rus, download=True)
deeppavlov_ner(document)

NameError: name 'build_model' is not defined

## 2.2. Написать свой нер попробовать разные подходы

a. передаём в сетку токен и его соседей

b. передаём в сетку только токен

c. свой вариант

In [37]:
!pip install razdel

In [38]:
from razdel import tokenize

In [39]:
words_docs = []
for ix, rec in enumerate(records):
    words = []
    for token in tokenize(rec.text):
        # if "http://" in input:
        #   token='None'
        type_ent = 'OUT'
        for ent in rec.spans:
            if (token.start >= ent.start) and (token.stop <= ent.stop):
                type_ent = ent.type
                break
        words.append([token.text, type_ent])
    words_docs.extend(words)

In [40]:
df_words = pd.DataFrame(words_docs, columns=['word', 'tag'])

In [41]:
df_words['tag'].value_counts()

OUT         219014
PER          21178
ORG          13641
LOC           4564
GEOPOLIT      4349
MEDIA         2481
Name: tag, dtype: int64

In [42]:
df_words.shape

(265227, 2)

In [43]:
!pip install tensorflow

  Using cached tensorflow-2.9.1-cp39-cp39-macosx_10_14_x86_64.whl (228.5 MB)


In [44]:
import tensorflow as tf

from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Embedding, GlobalAveragePooling1D, GlobalMaxPooling1D, Conv1D, GRU, LSTM, Dropout, Input, Bidirectional,Reshape
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization
from sklearn import model_selection, preprocessing, linear_model

In [45]:
train_x, valid_x, train_y, valid_y = model_selection.train_test_split(df_words['word'], df_words['tag'])

# labelEncode целевую переменную
encoder = preprocessing.LabelEncoder()
train_y = encoder.fit_transform(train_y)
valid_y = encoder.fit_transform(valid_y)

In [46]:
train_x.apply(len).max(axis=0)

55

In [47]:
train_data = tf.data.Dataset.from_tensor_slices((train_x, train_y))
valid_data = tf.data.Dataset.from_tensor_slices((valid_x, valid_y))

train_data = train_data.batch(2048)
valid_data = valid_data.batch(2048)

2022-06-16 20:06:55.987387: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [48]:
AUTOTUNE = tf.data.AUTOTUNE

train_data = train_data.cache().prefetch(buffer_size=AUTOTUNE)
valid_data = valid_data.cache().prefetch(buffer_size=AUTOTUNE)

In [49]:
def custom_standardization(input_data):
    return input_data

vocab_size = 30000
seq_len = 10


vectorize_layer = TextVectorization(
    standardize=custom_standardization,
    max_tokens=vocab_size,
    output_mode='int',
    # ngrams=(1, 3),
    output_sequence_length=seq_len
    )

# Make a text-only dataset (no labels) and call adapt to build the vocabulary.
text_data = train_data.map(lambda x, y: x)
vectorize_layer.adapt(text_data)

In [50]:
t=np.unique(encoder.inverse_transform(valid_y),return_counts=True)[1]
t=t/t.sum()
t  # распределение таргета

array([0.0167554 , 0.01749438, 0.00951634, 0.05053765, 0.82648891,
       0.07920732])

In [51]:
model = tf.keras.Sequential([
  vectorize_layer,
  tf.keras.layers.Embedding(len(vectorize_layer.get_vocabulary()), 64, mask_zero=True),
  tf.keras.layers.GlobalAveragePooling1D(),
  tf.keras.layers.Dense(300, activation='relu'),
  tf.keras.layers.Dropout(0.5),
  tf.keras.layers.Dense(50, activation='relu'),
  # tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(6, activation='softmax')
  ])

In [52]:
model.compile(optimizer='adam',         
              loss=tf.keras.losses.SparseCategoricalCrossentropy(),
              metrics=['accuracy'])
model.fit(train_data, validation_data=valid_data, epochs=3)

Epoch 1/3
98/98 [==============================] - 3s 22ms/step - loss: 0.7417 - accuracy: 0.8214 - val_loss: 0.4131 - val_accuracy: 0.8555
Epoch 2/3
98/98 [==============================] - 2s 21ms/step - loss: 0.2807 - accuracy: 0.9059 - val_loss: 0.2560 - val_accuracy: 0.9285
Epoch 3/3
98/98 [==============================] - 2s 23ms/step - loss: 0.1527 - accuracy: 0.9550 - val_loss: 0.2112 - val_accuracy: 0.9384


In [53]:
pred=model.predict(valid_data)

33/33 [==============================] - 0s 5ms/step


In [54]:
list(zip(encoder.classes_,pred.mean(axis=0)/t))

[('GEOPOLIT', 1.0001551459814169),
 ('LOC', 1.0322723401806735),
 ('MEDIA', 1.0034712958983523),
 ('ORG', 1.2441429276451785),
 ('OUT', 0.9872704747721538),
 ('PER', 0.9710172871872498)]

In [55]:
model = tf.keras.Sequential([
    vectorize_layer,
    tf.keras.layers.Embedding(len(vectorize_layer.get_vocabulary()), 64, mask_zero=True),
    # tf.keras.layers.Bidirectional(tf.keras.layers.GRU(64,  return_sequences=True)),
    tf.keras.layers.Bidirectional(tf.keras.layers.GRU(64)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(6,activation='softmax')
])

In [56]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(),
              metrics=['accuracy'])

In [57]:
model.fit(train_data, validation_data=valid_data, epochs=3)

Epoch 1/3
98/98 [==============================] - 20s 143ms/step - loss: 0.7987 - accuracy: 0.8244 - val_loss: 0.4123 - val_accuracy: 0.8542
Epoch 2/3
98/98 [==============================] - 13s 129ms/step - loss: 0.3159 - accuracy: 0.8992 - val_loss: 0.2821 - val_accuracy: 0.9138
Epoch 3/3
98/98 [==============================] - 12s 127ms/step - loss: 0.2068 - accuracy: 0.9367 - val_loss: 0.2363 - val_accuracy: 0.9326


## 2.3. Сравнить ваши реализованные подходы на качество (вывести precision/recall/f1_score)

In [63]:
%%time
crf = sklearn_crfsuite.CRF(
    algorithm='lbfgs',
    c1=0.1,
    c2=0.1,
    max_iterations=100,
    all_possible_transitions=True,
    verbose=True
)
crf.fit(train_data, valid_data)

loading training data to CRFsuite:   0%|                 | 0/98 [00:00<?, ?it/s]


TypeError: expected bytes, tensorflow.python.framework.ops.EagerTensor found

In [64]:
all_entities = sorted(df_words.entities.unique().tolist())
len(all_entities)

AttributeError: 'DataFrame' object has no attribute 'entities'

In [ ]:
# y_pred = crf.predict(X_test)
# metrics.flat_f1_score(y_test, y_pred, average='weighted', labels=all_entities)

In [ ]:
# y_pred = crf.predict(X_test)
# metrics.flat_f1_score(y_test, y_pred, average='weighted', labels=[i for i in all_entities if i != 'O'])

In [65]:
print(metrics.flat_classification_report(train_data, valid_data, labels = all_entities))

NameError: name 'all_entities' is not defined